# <p><center style="font-family:newtimeroman;font-size:180%;">Text Summarization</center></p>
### Table of contents:
* [Introduction](#0)
* [Import Libraries](#1)
* [Read Data](#2)
* [Extractive Summary](#3)
* [Abstractive Summary](#4)

**<a id="0"></a>
# <p style="background-image: url(https://i.postimg.cc/K87ByXmr/stage2.jpg);font-family:camtasia;font-size:120%;color:white;text-align:center;border-radius:15px 50px; padding:7px">Introduction  </p>
<a class="btn" href="#home">Tabel of Contents</a>

In extractive text summarization, the summary is created by selecting and extracting important sentences or phrases directly from the original text without any modification.

In abstractive text summarization, the model generates a summary by understanding the original text and paraphrasing it in a concise manner, possibly using new words or sentence structures.

**<a id="1"></a>
# <p style="background-image: url(https://i.postimg.cc/K87ByXmr/stage2.jpg);font-family:camtasia;font-size:120%;color:white;text-align:center;border-radius:15px 50px; padding:7px">Import Libraries </p>
<a class="btn" href="#home">Tabel of Contents</a>

In [1]:
import pandas as pd
import re 
import spacy
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from collections import defaultdict
from transformers import pipeline
from datasets import load_dataset
from collections import Counter
from heapq import nlargest
nltk.download('punkt')
nltk.download('stopwords')
import warnings
warnings.filterwarnings('ignore')

2024-08-17 21:50:14.226735: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-17 21:50:14.226847: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-17 21:50:14.418822: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


**<a id="2"></a>
# <p style="background-image: url(https://i.postimg.cc/K87ByXmr/stage2.jpg);font-family:camtasia;font-size:120%;color:white;text-align:center;border-radius:15px 50px; padding:7px">Read Data </p>
<a class="btn" href="#home">Tabel of Contents</a>

In [2]:
ds = load_dataset("knkarthick/dialogsum")
ds

Generating train split:   0%|          | 0/12460 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1500 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
})

In [3]:
data = pd.DataFrame(ds['train'])
data.head()

,id,dialogue,summary,topic
0,train_0,"#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. ...","Mr. Smith's getting a check-up, and Doctor Haw...",get a check-up
1,train_1,"#Person1#: Hello Mrs. Parker, how have you bee...",Mrs Parker takes Ricky for his vaccines. Dr. P...,vaccines
2,train_2,"#Person1#: Excuse me, did you see a set of key...",#Person1#'s looking for a set of keys and asks...,find keys
3,train_3,#Person1#: Why didn't you tell me you had a gi...,#Person1#'s angry because #Person2# didn't tel...,have a girlfriend
4,train_4,"#Person1#: Watsup, ladies! Y'll looking'fine t...",Malik invites Nikki to dance. Nikki agrees if ...,dance


In [4]:
text = data.dialogue[10]
text

"#Person1#: Could you do me a favor?\n#Person2#: Sure. What is it?\n#Person1#: Could you run over to the store? We need a few things.\n#Person2#: All right. What do you want me to get?\n#Person1#: Well, could you pick up some sugar?\n#Person2#: Okay. How much?\n#Person1#: A small bag. I guess we also need a few oranges.\n#Person2#: How many?\n#Person1#: Oh, let's see. . . About six.\n#Person2#: Anything else?\n#Person1#: Yes. We're out of milk.\n#Person2#: Okay. How much do you want me to get? A gallon?\n#Person1#: No. I think a half gallon will be enough.\n#Person2#: Is that all?\n#Person1#: I think so. Have you got all that?\n#Person2#: Yes. That's small bag of sugar, four oranges, and a half gallon of milk.\n#Person1#: Do you have enough money?\n#Person2#: I think so.\n#Person1#: Thanks very much. I appreciate it."

In [5]:
len(text)

809

In [6]:
data.summary[10]

'#Person1# asks #Person2# to do a favor. #Person2# agrees and helps buy a small bag of sugar, six oranges, and a half-gallon of milk.'

In [7]:
len(data.summary[10])

132

**<a id="3"></a>
# <p style="background-image: url(https://i.postimg.cc/K87ByXmr/stage2.jpg);font-family:camtasia;font-size:120%;color:white;text-align:center;border-radius:15px 50px; padding:7px">Extractive Summary </p>
<a class="btn" href="#home">Tabel of Contents</a>

In [8]:
def clean_text(text):
    # 1. Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

    # 2. Remove HTML tags
    text = re.sub(r'<.*?>', '', text)

    # 3. Remove Punctuation
    text = re.sub(r'[^\w\s]', '', text)

    # 4. Remove Numbers
    text = re.sub(r'\d+', '', text)

    # 5. Convert to Lowercase
    text = text.lower()

    # 6. Remove Stopwords (example stopwords list, customize as needed)
    stop_words = set(stopwords.words('english'))
    text = " ".join([word for word in text.split() if word not in stop_words])

    # 7. Remove Extra Whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [9]:
text = clean_text(text)
text

'person could favor person sure person could run store need things person right want get person well could pick sugar person okay much person small bag guess also need oranges person many person oh lets see six person anything else person yes milk person okay much want get gallon person think half gallon enough person person think got person yes thats small bag sugar four oranges half gallon milk person enough money person think person thanks much appreciate'

In [10]:
len(text)

461

In [11]:
nlp = spacy.load('en_core_web_sm')
doc = ""
def Selecting(text):
    global doc
    doc = nlp(text)
    tokens = []
    allowed_pos = ['ADJ','PROPN','VERB','NOUN']
    for token in doc :
        if token.pos_ in allowed_pos:
            tokens.append(token.text)
    return tokens
tokens = Selecting(text)
print(tokens)

['person', 'favor', 'person', 'sure', 'person', 'run', 'store', 'need', 'things', 'person', 'right', 'want', 'get', 'person', 'pick', 'sugar', 'person', 'much', 'person', 'small', 'bag', 'guess', 'need', 'oranges', 'person', 'many', 'person', 'lets', 'see', 'person', 'person', 'milk', 'person', 'want', 'get', 'gallon', 'person', 'think', 'gallon', 'enough', 'person', 'person', 'think', 'got', 'person', 's', 'small', 'bag', 'sugar', 'oranges', 'half', 'gallon', 'milk', 'person', 'enough', 'money', 'person', 'think', 'person', 'thanks', 'appreciate']


In [12]:
def Count(tokens):
    return Counter(tokens)
word_freq = Count(tokens)
print(word_freq)

Counter({'person': 19, 'gallon': 3, 'think': 3, 'need': 2, 'want': 2, 'get': 2, 'sugar': 2, 'small': 2, 'bag': 2, 'oranges': 2, 'milk': 2, 'enough': 2, 'favor': 1, 'sure': 1, 'run': 1, 'store': 1, 'things': 1, 'right': 1, 'pick': 1, 'much': 1, 'guess': 1, 'many': 1, 'lets': 1, 'see': 1, 'got': 1, 's': 1, 'half': 1, 'money': 1, 'thanks': 1, 'appreciate': 1})


In [13]:
def normalize_word_freq(word_freq):
    max_freq = max(word_freq.values())
    norm_word_freq = {word: freq / max_freq for word, freq in word_freq.items()}
    return norm_word_freq
norm_word_freq = normalize_word_freq(word_freq)
print(norm_word_freq)

{'person': 1.0, 'favor': 0.05263157894736842, 'sure': 0.05263157894736842, 'run': 0.05263157894736842, 'store': 0.05263157894736842, 'need': 0.10526315789473684, 'things': 0.05263157894736842, 'right': 0.05263157894736842, 'want': 0.10526315789473684, 'get': 0.10526315789473684, 'pick': 0.05263157894736842, 'sugar': 0.10526315789473684, 'much': 0.05263157894736842, 'small': 0.10526315789473684, 'bag': 0.10526315789473684, 'guess': 0.05263157894736842, 'oranges': 0.10526315789473684, 'many': 0.05263157894736842, 'lets': 0.05263157894736842, 'see': 0.05263157894736842, 'milk': 0.10526315789473684, 'gallon': 0.15789473684210525, 'think': 0.15789473684210525, 'enough': 0.10526315789473684, 'got': 0.05263157894736842, 's': 0.05263157894736842, 'half': 0.05263157894736842, 'money': 0.05263157894736842, 'thanks': 0.05263157894736842, 'appreciate': 0.05263157894736842}


In [14]:
def sentence_score(norm_word_freq):
    sent_token = [sent.text for sent in doc.sents]
    sent_score = {}
    for sent in sent_token:
        for word in word_tokenize(sent):
            if word.lower() in norm_word_freq:
                if sent not in sent_score.keys():
                    sent_score[sent] = norm_word_freq[word]
                else:
                    sent_score[sent] += norm_word_freq[word]
    return sent_score
score = sentence_score(norm_word_freq)
pd.DataFrame(list(score.items()),columns=['Sentence','Score'])

,Sentence,Score
0,person could favor person sure person could ru...,6.789474
1,okay much person small bag guess also need ora...,6.789474
2,okay much want get gallon person think half ga...,5.105263
3,thats small bag sugar four oranges half gallon...,4.210526


In [15]:
def largest(score,num_sentence = 2):
    sentence = nlargest(num_sentence,score,key = score.get)
    return ' '.join(sentence)
summary = largest(score)
summary

'person could favor person sure person could run store need things person right want get person well could pick sugar person okay much person small bag guess also need oranges person many person oh lets see six person anything else person yes milk person'

In [16]:
def Extractive_Summary(text,num_sentence=1):
    text = clean_text(text)
    tokens = Selecting(text)
    word_freq = Count(tokens)
    norm_word_freq = normalize_word_freq(word_freq)
    score = sentence_score(norm_word_freq)
    summary = largest(score,num_sentence)
    print('Summary:\n',summary)
Extractive_Summary(text)

Summary:
 person could favor person sure person could run store need things person right want get person well could pick sugar person


In [17]:
data.dialogue[50]

"#Person1#: You have the right to remain silent. Anything you say can and will be used against you in a court of law. You have the right to have an attorney present during questioning. If you cannot afford an attorney, one will be appointed for you. Do you understand?\n#Person2#: Yes.\n#Person1#: What's your name?\n#Person2#: My name is James.\n#Person1#: What's your nationality?\n#Person2#: American.\n#Person1#: What's your relationship with the victim?\n#Person2#: I don't know him.\n#Person1#: Why did you attack the victim?\n#Person2#: Because he beat me first when I tried to stop him from grabbing my bag and running away.\n#Person1#: How many times did you stab the victim?\n#Person2#: I stabbed his belly three times.\n#Person1#: Did you know that your actions might cause serous injuries or death?\n#Person2#: I knew, but I couldn't control myself.\n#Person1#: Was it your intention to kill the victim?\n#Person2#: No. I didn't kill him on purpose, madam. It's him who caused the inciden

In [18]:
Extractive_Summary(data.dialogue[50])

Summary:
 yes person whats name person name james person whats nationality person american person whats relationship victim person dont know person attack victim person beat first tried stop grabbing bag running away person many times stab victim person stabbed belly three times person know actions might cause serous injuries death person knew couldnt control person intention kill victim person didnt kill purpose madam caused incident need see attorney person ok give number well contact


**<a id="4"></a>
# <p style="background-image: url(https://i.postimg.cc/K87ByXmr/stage2.jpg);font-family:camtasia;font-size:120%;color:white;text-align:center;border-radius:15px 50px; padding:7px">Abstractive Summary </p>
<a class="btn" href="#home">Tabel of Contents</a>

In [19]:
def Abstractive_Summary(text):
    model = pipeline('summarization',model='t5-base',tokenizer='t5-base',framework='pt')
    summary = model(text,max_length=100,min_length=10,do_sample=False)
    print('Summary:\n',summary[0]['summary_text'])
Abstractive_Summary(text)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Your max_length is set to 100, but your input_length is only 89. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=44)


Summary:
 person could favor person sure person could run store need things person right want get person well could pick sugar person okay much person small bag guess also need oranges person many person oh lets see six person anything else person .


In [20]:
Abstractive_Summary(data.dialogue[50])

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Summary:
 #Person2#: I stabbed his belly three times . #Peter1#: i didn't kill him on purpose, madam. it's him who caused the incident.
